# ベースモデル

## 5/11 1600
StratifiedGraoupKfoldで安定的なCVの作成
validation loss = 0.31292   validation accuracy = 0.86842


In [1]:
#ColaboratoryかKaggleNotebookか判別
import sys
import os
from pathlib import Path

WORKING_DIR = '/content/drive/MyDrive/git/kaggle-inclass-brest-cancer-classification/working'  # コンペごとに書き換える
MAIN_INPUT_NAME = "brest-cancer-classification"     # コンペごとに書き換える

INPUT = Path('../input/')
MAIN_INPUT_ZIP = f'../input/{MAIN_INPUT_NAME}.zip'
if 'google.colab' in sys.modules:  # colab環境
    INPUT = Path('/content/input/')

    # drive mount
    from google.colab import drive
    drive.mount('/content/drive/')

    # working dir に移動
    os.chdir(WORKING_DIR)
    print("cwd:", os.getcwd())

    if not os.path.exists("/content/input"):
        # 必要な input を指定先へ unzip
        !unzip -q {MAIN_INPUT_ZIP} -d {INPUT}
        print("unzip INPUT")
    
elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    pass

!pip install timm
!pip install albumentations==0.4.5
!pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn==1.0.dev0

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
cwd: /content/drive/MyDrive/git/kaggle-inclass-brest-cancer-classification/working
Looking in indexes: https://pypi.org/simple, https://pypi.anaconda.org/scipy-wheels-nightly/simple


In [2]:
!nvidia-smi

Tue May 11 06:34:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## ライブラリ

In [3]:
import json
import os
import random
import time

import numpy as np
import pandas as pd
import cv2

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.cuda.amp import autocast, GradScaler

import timm
from tqdm.notebook import tqdm

from sklearn.model_selection import StratifiedKFold, GroupKFold, StratifiedGroupKFold
from sklearn.metrics import log_loss, f1_score

## 設定

In [14]:
class Config:
    def __init__(self, debug:bool):
        self.debug = debug 
        self.suffix = "_debug" if self.debug else ""
        self.data_directory = "./data"
        self.seed = 42

        self.fold_num = 3
        self.model_arch= "tf_efficientnet_b1"
        self.model_shape = "classifier"
        self.img_size_h= 50
        self.img_size_w= 50
        self.epochs = 10
        self.train_bs = 2048
        self.valid_bs = 2048
        self.T_0 = 10
        self.lr = 3e-3
        self.min_lr = 3e-5
        self.weight_decay = 1e-4
        self.num_workers = 4
        self.verbose_step = 1
        self.tta = 1
        self.monitor = "val_accuracy"
        self.patience = 3 # for EarlyStopping
        self.mode = "max"
        self.psudo = False
        self.device = "cuda:0"

config = Config(debug=False)
os.makedirs(config.data_directory, exist_ok=True)

## 便利関数

In [15]:
def seed_everything(seed):
    "seed値を一括指定"
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_img(path):
    """
    pathからimageの配列を得る
    """
    im_bgr = cv2.imread(path)
    if im_bgr is None:
        print(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

# データセット

In [16]:
class BreastDataset(Dataset):
    def __init__(self, df, transforms=None, output_label=True):

        super().__init__()
        self.df = df.copy()
        self.transforms = transforms
        self.output_label = output_label

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):
        if self.output_label:
            target = self.df["label"][index]

        img  = get_img(self.df.loc[index]["img_path"])

        if self.transforms:
            img = self.transforms(image=img)['image']

        if self.output_label:
            return img, target
        else:
            return img

def prepare_dataloader(df, input_shape, trn_idx, val_idx, train_bs, valid_bs, num_workers):
    train_ = df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)

    train_ds = BreastDataset(train_, transforms=get_train_transforms(input_shape), output_label=True)
    valid_ds = BreastDataset(valid_, transforms=get_valid_transforms(input_shape), output_label=True)

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=train_bs,
        pin_memory=True, # faster and use memory
        drop_last=False,
        shuffle=True,
        num_workers=num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        valid_ds,
        batch_size=valid_bs,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=True,
    )
    return train_loader, val_loader

# data augumentation

In [17]:
from albumentations import (
    PadIfNeeded, HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize,ToGray
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms(input_shape):
    return Compose([
            Resize(input_shape[0], input_shape[1]),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_valid_transforms(input_shape):
    return Compose([
                Resize(input_shape[0], input_shape[1]),
                Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
                ToTensorV2(p=1.0),
            ], p=1.)

def get_inference_transforms(input_shape):
    return Compose([
                Resize(input_shape[0], input_shape[1]),
                ShiftScaleRotate(p=0.5),
                ToTensorV2(p=1.0),
            ], p=1.)

# モデル

In [18]:
class BreastClassifier(nn.Module):
    def __init__(self, model_arch, n_class, model_shape, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)

        if model_shape == "classifier":
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, n_class)
        elif model_shape == "head":
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, n_class)
        elif model_shape == "fc":
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x

# 学習用

In [19]:
class EarlyStopping:
    def __init__(self, patience):
        self.max_val_monitor = 1000
        self.min_val_monitor = -1000
        self.val_epoch = -1
        self.stop_count = 0
        self.patience = patience
        self.min_delta = 0

    # mode = "min" or "max"(val_loss, (val_accuracy, val_f1score))
    def update(self, monitor, epoch, mode):
        if mode == "max":
            if monitor > self.min_val_monitor:
                self.min_val_monitor = monitor
                self.val_epoch = epoch
                self.stop_count = 0
            else:
                self.stop_count+=1
        else:
            if monitor < self.max_val_monitor:
                self.max_val_monitor = monitor
                self.val_epoch = epoch
                self.stop_count = 0
            else:
                self.stop_count+=1

        if self.stop_count >= self.patience:
            return -1
        else:
            return 0


def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device, verbose_step, scheduler=None, schd_batch_update=False):
    model.train()
    scaler = GradScaler()

    t = time.time()
    running_loss = None

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        with autocast():
            image_preds = model(imgs)
            loss = loss_fn(image_preds, image_labels)

            scaler.scale(loss).backward()

            if running_loss is None:
                running_loss = loss.item()
            else:
                running_loss = running_loss * .99 + loss.item() * .01

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None and schd_batch_update:
                scheduler.step()

            if ((step + 1) % verbose_step == 0) or ((step + 1) == len(train_loader)):
                description = f'train epoch {epoch} loss: {running_loss:.4f}'
                pbar.set_description(description)

    #print("train: "+ description)
    if scheduler is not None and not schd_batch_update:
        scheduler.step()

def valid_one_epoch(epoch, model, loss_fn, val_loader, device, verbose_step, scheduler=None, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    sample_num = 0
    image_preds_all = []
    image_targets_all = []

    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    for step, (imgs, image_labels) in pbar:
        imgs = imgs.to(device).float()
        image_labels = image_labels.to(device).long()

        image_preds = model(imgs)
        image_preds_all += [torch.argmax(image_preds, 1).detach().cpu().numpy()]
        image_targets_all += [image_labels.detach().cpu().numpy()]

        loss = loss_fn(image_preds, image_labels)

        loss_sum += loss.item()*image_labels.shape[0]
        sample_num += image_labels.shape[0]

        if ((step + 1) % verbose_step== 0) or ((step + 1) == len(val_loader)):
            description = f'valid epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)

    #print("valid "+ description)
    image_preds_all = np.concatenate(image_preds_all)
    image_targets_all = np.concatenate(image_targets_all)
    print('validation multi-class accuracy = {:.4f}'.format((image_preds_all==image_targets_all).mean()))

    if scheduler is not None:
        if schd_loss_update:
            scheduler.step(loss_sum/sample_num)
        else:
            scheduler.step()

    monitor = {}
    monitor["val_loss"] = loss_sum/sample_num
    monitor["val_accuracy"] = (image_preds_all==image_targets_all).mean()
    monitor["val_f1_score"] = f1_score(image_preds_all, image_targets_all)

    return monitor

def inference_one_epoch(model, data_loader, device):
    model.eval()
    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()

        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]

    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

# main


In [20]:
# train load

def load_train_df():
    base_path = os.path.join(INPUT, "train_image")
    df = pd.read_csv(os.path.join(INPUT, "train.csv"), index_col=0)
    df = pd.concat(
        [
          df,
          df['img_name'].str.rstrip(".png").str.split('_', expand=True).rename(columns={0: 'exam', 1: 'x', 2: 'y'}, inplace=False)
        ],
        axis=1
    )
    df["exam"] = df["exam"].astype(int)
    df["x"] = df["x"].str.lstrip("x_").astype(int)
    df["y"] = df["y"].str.lstrip("y_").astype(int)
    df["img_path"] = base_path + "/" + df["label"].astype(str).str.cat(df['img_name'], sep=os.sep)
    return df
train_df = load_train_df()
print(train_df.shape)
train_df.head()

(175889, 6)


,img_name,label,exam,x,y,img_path
0,8863_x2301_y1751.png,0,8863,2301,1751,/content/input/train_image/0/8863_x2301_y1751.png
1,8863_x1601_y751.png,0,8863,1601,751,/content/input/train_image/0/8863_x1601_y751.png
2,8863_x901_y1401.png,0,8863,901,1401,/content/input/train_image/0/8863_x901_y1401.png
3,8863_x651_y1101.png,0,8863,651,1101,/content/input/train_image/0/8863_x651_y1101.png
4,8863_x351_y651.png,0,8863,351,651,/content/input/train_image/0/8863_x351_y651.png


In [21]:
def load_test_df():
    base_path = os.path.join(INPUT, "test_image/test_image")
    df = pd.read_csv(os.path.join(INPUT, "submission.csv"))
    df = pd.concat(
        [
          df,
          df['Id'].str.rstrip(".png").str.split('_', expand=True).rename(columns={0: 'exam', 1: 'x', 2: 'y'}, inplace=False)
        ],
        axis=1
    )
    df["exam"] = df["exam"].astype(int)
    df["x"] = df["x"].str.lstrip("x_").astype(int)
    df["y"] = df["y"].str.lstrip("y_").astype(int)
    df["img_path"] = base_path + "/" +  df['Id']
    return df

test_df = load_test_df()
print(test_df.shape)
test_df.head()

(101635, 6)


,Id,label,exam,x,y,img_path
0,9381_x951_y601.png,0,9381,951,601,/content/input/test_image/test_image/9381_x951...
1,9381_x951_y2651.png,0,9381,951,2651,/content/input/test_image/test_image/9381_x951...
2,9381_x951_y2001.png,0,9381,951,2001,/content/input/test_image/test_image/9381_x951...
3,9381_x951_y1951.png,0,9381,951,1951,/content/input/test_image/test_image/9381_x951...
4,9381_x951_y1901.png,0,9381,951,1901,/content/input/test_image/test_image/9381_x951...


In [23]:

train = load_train_df()
if config.debug:
  train = train.sample(n=100).reset_index(drop=True)
test = load_test_df()


seed_everything(config.seed)
device = torch.device(config.device)
val_epochs = []

# config parameter
input_shape = (config.img_size_h, config.img_size_w)
train_bs=config.train_bs
valid_bs=config.valid_bs
num_workers=config.num_workers


folds = StratifiedGroupKFold(n_splits=config.fold_num, shuffle=True, random_state=config.seed).split(np.arange(train.shape[0]), train.label.values, train["exam"].values)
#folds = GroupKFold(n_splits=config.fold_num).split(np.arange(train.shape[0]), train.label.values, train["exam"].values)
#folds = StratifiedKFold(n_splits=config.fold_num, shuffle=True, random_state=config.seed).split(np.arange(train.shape[0]), train.label.values)
for fold, (trn_idx, val_idx) in enumerate(folds):
    '''
    if fold > 0: # 時間がかかるので最初のモデルのみ
        break
    '''

    print(f'Training with fold {fold} started (train:{len(trn_idx)}, val:{len(val_idx)})')

    train_loader, val_loader = prepare_dataloader(train, 
                                                  input_shape,
                                                  trn_idx, val_idx,
                                                  train_bs, valid_bs,
                                                  num_workers)

    model = BreastClassifier(config.model_arch, 
                              train.label.nunique(),
                              config.model_shape,
                              pretrained=True).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=config.T_0, T_mult=1, eta_min=config.min_lr, last_epoch=-1)
    er = EarlyStopping(config.patience)

    loss_tr = nn.CrossEntropyLoss().to(device)
    loss_fn = nn.CrossEntropyLoss().to(device)

    for epoch in range(config.epochs):
        train_one_epoch(epoch, model, 
                        loss_tr, optimizer, 
                        train_loader, device, 
                        config.verbose_step,
                        scheduler=scheduler, schd_batch_update=False)

        with torch.no_grad():
            monitor = valid_one_epoch(epoch, model, 
                                      loss_fn, val_loader, 
                                      device, config.verbose_step,
                                      scheduler=None, schd_loss_update=False)

        # Early Stopiing
        if er.update(monitor[config.monitor], epoch, config.mode) < 0:
            break

        if epoch == er.val_epoch:
            model_path = f'{config.data_directory}/{config.model_arch}_fold_{fold}_{epoch}.pt'
            print(f"saved {model_path}")
            torch.save(model.state_dict(), model_path)

    val_epochs.append(er.val_epoch)
    del model, optimizer, train_loader, val_loader,  scheduler
    torch.cuda.empty_cache()



#--- infer
tst_preds = []
val_loss = []
val_acc = []

seed_everything(config.seed)
folds = StratifiedGroupKFold(n_splits=config.fold_num, shuffle=True, random_state=config.seed).split(np.arange(train.shape[0]), train.label.values, train["exam"].values)
print("val_epochs : ", val_epochs)
for fold, (trn_idx, val_idx) in enumerate(folds):
    '''
    if fold > 0: # 時間がかかるので最初のモデルのみ
        break
    '''
    print(' fold {} started'.format(fold))

    valid_ = train.loc[val_idx,:].reset_index(drop=True)
    valid_ds = BreastDataset(valid_, transforms=get_valid_transforms(input_shape), output_label=False)

    test_ds = BreastDataset(test_df, transforms=get_valid_transforms(input_shape), output_label=False)

    val_loader = torch.utils.data.DataLoader(
            valid_ds,
            batch_size=valid_bs,
            num_workers=num_workers,
            shuffle=False,
            pin_memory=False,
        )

    tst_loader = torch.utils.data.DataLoader(
        test_ds,
        batch_size=valid_bs,
        num_workers=num_workers,
        shuffle=False,
        pin_memory=False,
    )

    device = torch.device(config.device)
    model = BreastClassifier(config.model_arch, train.label.nunique(), config.model_shape).to(device)

    val_preds = []

    model_path = f'{config.data_directory}/{config.model_arch}_fold_{fold}_{val_epochs[fold]}.pt'
    model.load_state_dict(torch.load(model_path))

    with torch.no_grad():
        val_preds += [inference_one_epoch(model, val_loader, device)]
        tst_preds += [inference_one_epoch(model, tst_loader, device)]

    val_preds = np.mean(val_preds, axis=0)
    val_loss.append(log_loss(valid_.label.values, val_preds))
    val_acc.append((valid_.label.values == np.argmax(val_preds, axis=1)).mean())

    del model, device, val_loader, tst_loader
    print()
print(f'validation loss = {np.mean(val_loss):.5f}   validation accuracy = {np.mean(val_acc):.5f}')
tst_preds = np.mean(tst_preds, axis=0)

torch.cuda.empty_cache()
tst_preds_label_all = np.argmax(tst_preds, axis=1)

# 予測結果を保存
sub = pd.read_csv(os.path.join(INPUT, "submission.csv"))
sub['label'] = tst_preds_label_all

sub.to_csv('submission.csv', index=False)

Training with fold 0 started (train:118023, val:57866)



validation multi-class accuracy = 0.8472
saved ./data/tf_efficientnet_b1_fold_0_0.pt



validation multi-class accuracy = 0.8542
saved ./data/tf_efficientnet_b1_fold_0_1.pt



validation multi-class accuracy = 0.8156



validation multi-class accuracy = 0.8422


...


validation multi-class accuracy = 0.8818



validation multi-class accuracy = 0.8778
val_epochs :  [1, 4, 2]
 fold 0 started



 fold 1 started



 fold 2 started



validation loss = 0.31292   validation accuracy = 0.86842
